In [ ]:
import pandas as pd
import numpy as np
import ast


In [ ]:
df = pd.read_excel('/content/kaggle_q05_science_test_20230403_all_attribution_scores (1).xlsx')

df = df[(df.is_deberta_v2_pred_same_as_true_score ==True) & (df.is_bert_pred_same_as_true_score ==True)]

df = df[df.essay_score != 0]
vals = ['Id', 'attrib_lime_deberta_rW+_tok_seed_0_sample_1000',
           "attrib_loo_deberta_rW+_tok",'attrib_ig_deberta',
           "attrib_hedge_deberta_20250811",     'attrib_loo_bert_rW+_tok',
    'attrib_lime_bert_rW+_tok_seed_0_sample_1000',
    'attrib_ig_bert',
    'attrib_hedge_bert_20250812', 'essay_score', 'EssayText',
        "AOPC_lime_deberta_rW+_tok_seed_0_sample_1000", "AOPC_loo_deberta_rW+_tok", "AOPC_ig_deberta",
        "AOPC_hedge_deberta_20250811", 'AOPC_loo_bert_rW+_tok', 'AOPC_lime_bert_rW+_tok_seed_0_sample_1000', 'AOPC_ig_bert_rW+_tok',
        "AOPC_hedge_bert_20250812"

        ]
df = df[vals].dropna()

# preprocessing
df['attrib_hedge_bert_20250812'] = df['attrib_hedge_bert_20250812'].apply(ast.literal_eval)
df['attrib_hedge_deberta_20250811'] = df['attrib_hedge_deberta_20250811'].apply(ast.literal_eval)

def sorting(data):
  data= sorted(data, key=lambda x: x[2][0])
  cleaned_data = [(x[0], x[1]) for x in data]
  return cleaned_data
df['attrib_hedge_bert_20250812'] =df['attrib_hedge_bert_20250812'].apply(sorting)
df['attrib_hedge_deberta_20250811'] = df['attrib_hedge_deberta_20250811'].apply(sorting)



## Visualization

In [ ]:
import pandas as pd
from IPython.display import HTML
import ast
import re

def highlight_with_unique_match_colors(df, columns, essay_col='EssayText', top_percent=0.2):
    font_colors = ['#B22222', '#1E90FF', '#228B22', '#8B008B',
                   '#A0522D', '#5F9EA0', '#DAA520', '#4B0082']

    def normalize_token(tok):
        return re.sub(r'\W+', '', tok.lower())

    def process_row(row):
        top_tokens_by_col = {}
        all_top_norm_tokens = []

        # Step 1: Extract and rank top tokens per column
        for col in columns:
            cell = row[col]
            if isinstance(cell, str):
                try:
                    cell = ast.literal_eval(cell)
                except:
                    cell = []

            token_scores = list(cell.items()) if isinstance(cell, dict) else cell
            sorted_tokens = sorted(token_scores, key=lambda x: x[1], reverse=True)  # use raw score
            top_n = max(1, int(len(sorted_tokens) * top_percent))

            if "attrib_ig_bert" in col:
                top_pairs = sorted_tokens[:top_n]
                norm_top_tokens = [normalize_token(tok) for tok, _ in sorted_tokens[:top_n]]
                top_tokens_by_col[col] = {
                    "tokens": token_scores,
                    "top_pairs": top_pairs,
                    "norm_top_tokens": norm_top_tokens
                }
                all_top_norm_tokens.extend(norm_top_tokens)
            elif "hedge" in col:
                top_pairs = sorted_tokens[:top_n]
                norm_top_tokens = [normalize_token(tok) for tok, _ in sorted_tokens[:top_n]]
                top_tokens_by_col[col] = {
                    "tokens": token_scores,
                    "top_pairs": top_pairs,
                    "norm_top_tokens": norm_top_tokens
                }
                all_top_norm_tokens.extend(norm_top_tokens)
            else:
                norm_top_tokens = [normalize_token(tok) for tok, _ in sorted_tokens[:top_n]]
                top_tokens_by_col[col] = {
                    "tokens": token_scores,
                    "top_set": set(norm_top_tokens)
                }
                all_top_norm_tokens.extend(norm_top_tokens)

        # Step 2: Find overlapping normalized tokens
        match_counts = pd.Series(all_top_norm_tokens).value_counts()
        matched_tokens = [tok for tok, count in match_counts.items() if count >= 2]
        match_color_map = {tok: font_colors[i % len(font_colors)] for i, tok in enumerate(matched_tokens)}

        # Step 3: Render each column
        styled_row = {}
        for col in columns:
            styled_tokens = []

            if "attrib_ig_bert" in col:
                token_scores = top_tokens_by_col[col]["tokens"]
                top_pairs = top_tokens_by_col[col]["top_pairs"]

                combined_word = ""
                building_word = False

                for i, (tok, score) in enumerate(token_scores):
                    if tok in ['[CLS]', '[SEP]']:
                        continue  # ✅ skip [CLS] and [SEP] in visualization

                    norm_tok = normalize_token(tok)
                    display_tok = tok[2:] if tok.startswith("##") else tok
                    is_top = (tok, score) in top_pairs
                    color = match_color_map.get(norm_tok, None)

                    styled_tok = display_tok
                    if is_top:
                        if color:
                            styled_tok = f"<span style='background-color: #FFF9C4; color: {color}; font-weight: bold'>{display_tok}</span>"
                        else:
                            styled_tok = f"<span style='background-color: #FFF9C4'>{display_tok}</span>"

                    if tok.startswith("##"):
                        combined_word += styled_tok
                        building_word = True
                    else:
                        if combined_word:
                            styled_tokens.append(combined_word)
                            combined_word = ""
                        spacer = "" if len(styled_tokens) == 0 else " "
                        combined_word = spacer + styled_tok
                        building_word = True

                if combined_word:
                    styled_tokens.append(combined_word)

                styled_row[col] = "".join(styled_tokens)
            elif "hedge" in col:
                token_scores = top_tokens_by_col[col]["tokens"]
                top_pairs = top_tokens_by_col[col]["top_pairs"]

                combined_word = ""
                building_word = False

                for i, (tok, score) in enumerate(token_scores):
                    if tok in ['[CLS]', '[SEP]']:
                        continue  # ✅ skip [CLS] and [SEP] in visualization

                    norm_tok = normalize_token(tok)
                    display_tok = tok[2:] if tok.startswith("##") else tok
                    is_top = (tok, score) in top_pairs
                    color = match_color_map.get(norm_tok, None)

                    styled_tok = display_tok
                    if is_top:
                        if color:
                            styled_tok = f"<span style='background-color: #FFF9C4; color: {color}; font-weight: bold'>{display_tok}</span>"
                        else:
                            styled_tok = f"<span style='background-color: #FFF9C4'>{display_tok}</span>"

                    if tok.startswith("##"):
                        combined_word += styled_tok
                        building_word = True
                    else:
                        if combined_word:
                            styled_tokens.append(combined_word)
                            combined_word = ""
                        spacer = "" if len(styled_tokens) == 0 else " "
                        combined_word = spacer + styled_tok
                        building_word = True

                if combined_word:
                    styled_tokens.append(combined_word)

                styled_row[col] = "".join(styled_tokens)

            else:
                essay_tokens = row[essay_col].split()
                top_set = top_tokens_by_col[col]["top_set"]

                for tok in essay_tokens:
                    norm_tok = normalize_token(tok)
                    if norm_tok in top_set:
                        if norm_tok in match_color_map:
                            styled_tokens.append(
                                f"<span style='background-color: #FFF9C4; color: {match_color_map[norm_tok]}; font-weight: bold'>{tok}</span>"
                            )
                        else:
                            styled_tokens.append(f"<span style='background-color: #FFF9C4'>{tok}</span>")
                    else:
                        styled_tokens.append(tok)

                styled_row[col] = " ".join(styled_tokens)

        return pd.Series(styled_row)

    return df.apply(process_row, axis=1)

def render_html_table(df, column_widths=None, default_width='500px'):
    html = "<table border='1' style='border-collapse: collapse; table-layout: fixed;'>"

    # Table headers
    html += "<tr>"
    for col in df.columns:
        width = column_widths.get(col, default_width) if column_widths else default_width
        html += f"<th style='width:{width}; word-wrap: break-word'>{col}</th>"
    html += "</tr>"

    # Table rows
    for _, row in df.iterrows():
        html += "<tr>"
        for col in df.columns:
            width = column_widths.get(col, default_width) if column_widths else default_width
            html += f"<td style='width:{width}; word-wrap: break-word; vertical-align: top'>{row[col]}</td>"
        html += "</tr>"

    html += "</table>"
    return HTML(html)


In [ ]:
# Define nice display names
columns = ['attrib_lime_deberta_rW+_tok_seed_0_sample_1000',
           "attrib_loo_deberta_rW+_tok",
           'attrib_ig_deberta',
           "attrib_hedge_deberta_20250811"]
column_display_names = {
    'attrib_lime_deberta_rW+_tok_seed_0_sample_1000': 'LIME (DeBERTa)',
    "attrib_loo_deberta_rW+_tok": 'Leave-One-Out (DeBERTa)',
    'attrib_ig_deberta': 'Integrated Gradients (DeBERTa)',
    "attrib_hedge_deberta_20250811": "HEDGE (DeBERTa)"
}

subset = df#[df.Id.isin([13173, 13015])]
# Apply highlight function
highlighted_df = highlight_with_unique_match_colors(subset, columns, essay_col='EssayText')

# Add essay score column and rename
highlighted_df['Score'] = subset['essay_score'].values
highlighted_df = highlighted_df[['Score'] + list(highlighted_df.columns[:-1])]
highlighted_df = highlighted_df.rename(columns=column_display_names)

highlighted_df['AOPC LIME'] = subset['AOPC_lime_deberta_rW+_tok_seed_0_sample_1000'].round(3).values
highlighted_df['AOPC LOO'] = subset['AOPC_loo_deberta_rW+_tok'].round(3).values
highlighted_df['AOPC IG'] = subset['AOPC_ig_deberta'].round(3).values
highlighted_df['AOPC HEDGE'] = subset['AOPC_hedge_deberta_20250811'].round(3).values
highlighted_df = highlighted_df.sort_values('Score')

# Define column widths
column_widths = {
    'Score': '30px',  # Narrower column
    'LIME (DeBERTa)': '500px',
    'Integrated Gradients (DeBERTa)': '500px',
    'Leave-One-Out (DeBERTa)': '500px',
    'HEDGE (DeBERTa)': '500px',
    'AOPC LIME': '15px',
    'AOPC LOO': '15px',
    'AOPC IG': '15px',
    'AOPC HEDGE': '15px'
}

# Render with customized widths
render_html_table(highlighted_df, column_widths=column_widths)


Score,LIME (DeBERTa),Leave-One-Out (DeBERTa),Integrated Gradients (DeBERTa),HEDGE (DeBERTa),AOPC LIME,AOPC LOO,AOPC IG,AOPC HEDGE
1,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid. The codons match up It creates protein.,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid. The codons match up It creates protein.,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid. The codons match up It creates protein.,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid . The codon s match up It creates protein .,0.474,0.381,0.232,0.142
1,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body.,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body.,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body.,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body .,0.928,0.927,0.923,0.086
1,"The mRNA then goes into the ribosome to be transcript. Other pieces of rRNA are attached to the ribosome. A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip. When this process is done, proteins have been formed, and are ready to perform their jobs.","The mRNA then goes into the ribosome to be transcript. Other pieces of rRNA are attached to the ribosome. A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip. When this process is done, proteins have been formed, and are ready to perform their jobs.","The mRNA then goes into the ribosome to be transcript. Other pieces of rRNA are attached to the ribosome. A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip. When this process is done, proteins have been formed, and are ready to perform their jobs.","The mRNA then goes into the ribosome to be transcript . Other pieces of rRNA are attached to the ribosome . A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip . When this process is done , proteins have been formed , and are ready to perform their jobs .",0.825,0.494,0.015,0.011
1,"First the mRNA leaves the nucleus, then it goes between a ribosome, then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for.","First the mRNA leaves the nucleus, then it goes between a ribosome, then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for.","First the mRNA leaves the nucleus, then it goes between a ribosome, then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for.","First the mRNA leaves the nucleus , then it goes between a ribosome , then it transcribe s into rRNA to create polypeptide s amino acids that will serve out there duty to what they were made for .",0.721,0.236,0.835,0.014
1,"The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly. The rRNA then codes what the mRNA said to code, which of course then goes to the tRNA. The tRNA then brings the RNA strand to fuse with another RNA strand. Thus, the two strands become a DNA strand.","The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly. The rRNA then codes what the mRNA said to code, which of course then goes to the tRNA. The tRNA then brings the RNA strand to fuse with another RNA strand. Thus, the two strands become a DNA strand.","The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly. The rRNA then codes what the mRNA sai

In [ ]:
# Replace with your actual DataFrame and column names
columns = [
    'attrib_loo_bert_rW+_tok',
    'attrib_lime_bert_rW+_tok_seed_0_sample_1000',
    'attrib_ig_bert',
    'attrib_hedge_bert_20250812'
]

column_display_names = {
    'attrib_loo_bert_rW+_tok': 'Leave-One-Out (BERT)',
    "attrib_lime_bert_rW+_tok_seed_0_sample_1000": 'Lime (BERT)',
    'attrib_ig_bert': 'Integrated Gradients (BERT)',
    "attrib_hedge_bert_20250812": "HEDGE (BERT)"
}



subset = df#[df.Id.isin([13291, 13041, 12770])]

# Apply highlight function
highlighted_df = highlight_with_unique_match_colors(subset, columns, essay_col='EssayText')
highlighted_df['Score'] = subset['essay_score'].values
highlighted_df = highlighted_df[['Score'] + list(highlighted_df.columns[:-1])]
highlighted_df = highlighted_df.rename(columns=column_display_names)

highlighted_df['AOPC LOO'] = subset['AOPC_loo_bert_rW+_tok'].round(3).values
highlighted_df['AOPC LIME'] = subset['AOPC_lime_bert_rW+_tok_seed_0_sample_1000'].round(3).values
highlighted_df['AOPC IG'] = subset['AOPC_ig_bert_rW+_tok'].round(3).values
highlighted_df['AOPC HEDGE'] = subset['AOPC_hedge_bert_20250812'].round(3).values
highlighted_df =highlighted_df.sort_values('Score')

# Define column widths
column_widths = {
    'Score': '30px',  # Narrower column
    'LIME (DeBERTa)': '500px',
    'Integrated Gradients (DeBERTa)': '500px',
    'AOPC LOO': '15px',
    'AOPC LIME': '15px',
    'AOPC IG': '15px',
    'AOPC HEDGE': '15px'
}

# Render with customized widths
render_html_table(highlighted_df, column_widths=column_widths)


Score,Leave-One-Out (BERT),Lime (BERT),Integrated Gradients (BERT),HEDGE (BERT),AOPC LOO,AOPC LIME,AOPC IG,AOPC HEDGE
1,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid. The codons match up It creates protein.,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid. The codons match up It creates protein.,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid . The codons match up It creates protein .,tRNA transfers it outside the nucleus to be matched with a ribosome The ribosome latches on to an amino acid . The codons match up It creates protein .,0.713,0.709,0.896,0.897
1,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body.,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body.,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body .,mRNA leaves the nucleus to the ribosome where it makes copies of itself to distribute throughout the body .,0.904,0.905,0.905,0.904
1,"The mRNA then goes into the ribosome to be transcript. Other pieces of rRNA are attached to the ribosome. A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip. When this process is done, proteins have been formed, and are ready to perform their jobs.","The mRNA then goes into the ribosome to be transcript. Other pieces of rRNA are attached to the ribosome. A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip. When this process is done, proteins have been formed, and are ready to perform their jobs.","The mRNA then goes into the ribosome to be transcript . Other pieces of rRNA are attached to the ribosome . A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip . When this process is done , proteins have been formed , and are ready to perform their jobs .","The mRNA then goes into the ribosome to be transcript . Other pieces of rRNA are attached to the ribosome . A specific enzyme is attached and starts to pair up the mRNA with other RNA to create a protein strip . When this process is done , proteins have been formed , and are ready to perform their jobs .",0.627,0.834,0.771,0.893
1,"First the mRNA leaves the nucleus, then it goes between a ribosome, then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for.","First the mRNA leaves the nucleus, then it goes between a ribosome, then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for.","First the mRNA leaves the nucleus , then it goes between a ribosome , then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for .","First the mRNA leaves the nucleus , then it goes between a ribosome , then it transcribes into rRNA to create polypeptides amino acids that will serve out there duty to what they were made for .",0.057,0.533,0.905,0.906
1,"The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly. The rRNA then codes what the mRNA said to code, which of course then goes to the tRNA. The tRNA then brings the RNA strand to fuse with another RNA strand. Thus, the two strands become a DNA strand.","The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly. The rRNA then codes what the mRNA said to code, which of course then goes to the tRNA. The tRNA then brings the RNA strand to fuse with another RNA strand. Thus, the two strands become a DNA strand.","The mRNA goes off to the rRNA stage to deliver the message of what the rRNA needs to code exactly . The rRNA then codes what the mRNA said 